In [10]:
import io
import sys

from google.cloud import storage

sys.path.append("..")
import os
from pathlib import Path

from dotenv import load_dotenv

from function.cloud import list_files
from function.retriever import initialize_retriever

retriever = initialize_retriever()

metadatas = [entry.metadata for entry in retriever.vectorstore.docstore._dict.values()]
all_stored_venues = set(_["company"] for _ in metadatas)
print("stored venues: ", len(all_stored_venues))
all_venues = set([Path(path).stem for path in list_files(filter=r"venues/")])

unprocessed_venues = all_venues - all_stored_venues
venue = list(unprocessed_venues)
print("unprocessed venues: ", len(venue))

Fetching database from the cloud...
Downloaded 2 files from '<Blob: wedding-venues-001, faiss_db, None>' to /Users/lanchu/Documents/projects/001-wedding-venue/noteboobs/faiss_db
Loaded FAISS index from /Users/lanchu/Documents/projects/001-wedding-venue/noteboobs/faiss_db
stored venues:  272
unprocessed venues:  9


In [6]:
unprocessed_venues

{'Avenue of the Arts',
 'Flying Flags Resort',
 'La Jolla Beach & Tennis Club',
 'Las Casuelas Nuevas',
 'Muckenthaler Mansion',
 'Pala Mesa Resort',
 'Park Club Costa Mesa',
 'The Grand Long Beach',
 'Vibiana'}

In [7]:
load_dotenv(override=True)

True

In [9]:
import sys

sys.path.append("..")

from function.retriever import load_venue_metadata, preprocess_documents

venue_metadata = load_venue_metadata()
info = preprocess_documents(
    ["Pala Mesa Resort", "La Jolla Beach & Tennis Club"], venue_metadata
)
# PERSIST_DIRECTORY: str = os.getenv("DATABASE_DIR")

# add_documents_to_retriever(info, retriever, venue_metadata)
# retriever.vectorstore.save_local(PERSIST_DIRECTORY)

Fetching database from the cloud...
Downloaded 2 files from '<Blob: wedding-venues-001, faiss_db, None>' to /Users/lanchu/Documents/projects/001-wedding-venue/noteboobs/faiss_db
Loaded FAISS index from /Users/lanchu/Documents/projects/001-wedding-venue/noteboobs/faiss_db
Fetching database from the cloud...
Loaded FAISS index from /Users/lanchu/Documents/projects/001-wedding-venue/noteboobs/faiss_db
searching for Pala Mesa Resort.pdf on Google Cloud...
downloading Pala Mesa Resort.pdf from Google Cloud...
sending Pala Mesa Resort.pdf to Adobe...
error with Adobe operation: description =Request could not be completed. Possible cause attached!, requestTrackingId=(<class 'requests.exceptions.ConnectionError'>, ConnectionError(ProtocolError('Connection aborted.', TimeoutError('The write operation timed out'))), <traceback object at 0x31330ba00>)
Fetching database from the cloud...
Loaded FAISS index from /Users/lanchu/Documents/projects/001-wedding-venue/noteboobs/faiss_db
searching for La 

In [6]:
from function.retriever import initialize_retriever

retriever = initialize_retriever()
texts = [
    doc.metadata["company"]
    for doc in filter(
        lambda doc: doc.metadata["type"] == "text",
        retriever.vectorstore.docstore._dict.values(),
    )
]
len(texts)

# for doc in retriever.vectorstore.docstore._dict.values():
#     print(doc.metadata["company"], doc.metadata["type"], doc.metadata.get("image_path"))

Fetching database from the cloud...
Downloaded 2 files from '<Blob: wedding-venues-001, faiss_db, None>' to /Users/lanchu/Documents/projects/001-wedding-venue/noteboobs/faiss_db
Loaded FAISS index from /Users/lanchu/Documents/projects/001-wedding-venue/noteboobs/faiss_db


96

In [9]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

from function.secrets import secrets

embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=secrets.OPENAI_API_KEY.get_secret_value(),
)
# if not os.path.exists(PERSIST_DIRECTORY):
# print("Fetching database from the cloud...")
# download_faiss_from_cloud(bucket.blob("faiss_db"), PERSIST_DIRECTORY)
# Try to load

vectorstore = FAISS.load_local(
    "faiss_db", embedding_model, allow_dangerous_deserialization=True
)

In [10]:
PERSIST_DIRECTORY = "notebooks/faiss_db"
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=os.getenv("OPENAI_API_KEY"),
)
vectorstore = FAISS.load_local(
    PERSIST_DIRECTORY, embedding_model, allow_dangerous_deserialization=True
)
vectorstore.docstore._dict

{'e1423160-d77f-4b92-a60b-71c997fcd402': Document(metadata={'company': 'dummy', 'type': 'text'}, page_content='dummy document')}